<a href="https://colab.research.google.com/github/piyumalanthony/CapsNet_Implementation/blob/master/tf_Sinhala_caps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Capsule/Sinhala Capsnet

/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Capsule/Sinhala Capsnet


In [ ]:
!pip install tensorflow==1.8.0

     |████████████████████████████████| 49.1MB 83kB/s 
     |████████████████████████████████| 3.1MB 45.6MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=db2e00d48a80aaefdc74e6e5f4d6e26a7608364c602bbdf4f8de8f4a2332be8b
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
!pip install keras==2.1.5

     |████████████████████████████████| 337kB 2.7MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
import h5py
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import keras
from keras import backend as K
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [ ]:
# import tensorflow.compat.v1 as tf
# from tensorflow.compat.v1 import initializers
# tf.disable_v2_behavior()
import tensorflow as tf

In [ ]:
# init = tf.initializers.GlorotUniform()
# var = tf.Variable(init(shape=shape))
# # or a oneliner with a little confusing brackets
# var = tf.Variable(tf.initializers.GlorotUniform()(shape=shape))
print(tf.__version__)

1.8.0


In [ ]:
from tensorflow.contrib.layers.python.layers import initializers


# **Model**


In [ ]:
def _matmul_broadcast(x, y, name):
  """Compute x @ y, broadcasting over the first `N - 2` ranks.
  """
  with tf.variable_scope(name) as scope:
    return tf.reduce_sum(
      tf.nn.dropout(x[..., tf.newaxis] * y[..., tf.newaxis, :, :],1), axis=-2
    )


In [ ]:
z = tf.constant(0.1, shape=[2,3,4])

In [ ]:
z[...,tf.newaxis]

<tf.Tensor 'strided_slice:0' shape=(2, 3, 4, 1) dtype=float32>

In [ ]:
def _conv2d_wrapper(inputs, shape, strides, padding, add_bias, activation_fn, name, stddev=0.1):
  """Wrapper over tf.nn.conv2d().
  """

  with tf.variable_scope(name) as scope:
    kernel = _get_weights_wrapper(
      name='weights', shape=shape, weights_decay_factor=0.0, )
    output = tf.nn.conv2d(inputs, filter=kernel, strides=strides, padding=padding, name='conv')
    if add_bias:
      biases = _get_biases_wrapper(name='biases', shape=[shape[-1]] )
      output = tf.add(output, biases, name='biasAdd')
    if activation_fn is not None:
      output = activation_fn(output, name='activation')
  return output

In [ ]:
def _get_weights_wrapper(name, shape, dtype=tf.float32, initializer=initializers.xavier_initializer(),weights_decay_factor=None):
  """Wrapper over _get_variable_wrapper() to get weights, with weights decay factor in loss.
  """

  weights = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)

  if weights_decay_factor is not None and weights_decay_factor > 0.0:
    weights_wd = tf.multiply(tf.nn.l2_loss(weights), weights_decay_factor, name=name + '/l2loss')
    tf.add_to_collection('losses', weights_wd)

  return weights

In [ ]:
def _get_biases_wrapper(name, shape, dtype=tf.float32, initializer=tf.constant_initializer(0.0)):
  """Wrapper over _get_variable_wrapper() to get bias.
  """
  biases = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)
  return biases

In [ ]:
def _get_variable_wrapper(
  name, shape=None, dtype=None, initializer=None,
  regularizer=None,
  trainable=True,
  collections=None,
  caching_device=None,
  partitioner=None,
  validate_shape=True,
  custom_getter=None
):
  """Wrapper over tf.get_variable().
  """

  with tf.device('/cpu:0'):
    var = tf.get_variable(
      name, shape=shape, dtype=dtype, initializer=initializer,
      regularizer=regularizer, trainable=trainable,
      collections=collections, caching_device=caching_device,
      partitioner=partitioner, validate_shape=validate_shape,
      custom_getter=custom_getter
    )
  return var

In [ ]:
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)

In [ ]:
def squash_v1(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    return scale * x

In [ ]:
def squash_v0(s, axis=-1, epsilon=1e-7, name=None):
    s_squared_norm = K.sum(K.square(s), axis, keepdims=True) + K.epsilon()
    safe_norm = K.sqrt(s_squared_norm)
    scale = 1 - tf.exp(-safe_norm)
    return scale * s / safe_norm

In [ ]:
def routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations):
    b = keras.backend.zeros_like(u_hat_vecs[:,:,:,0])
    if i_activations is not None:
        i_activations = i_activations[...,tf.newaxis]
    for i in range(iterations):
        if False:
            leak = tf.zeros_like(b, optimize=True)
            leak = tf.reduce_sum(leak, axis=1, keep_dims=True)
            leaky_logits = tf.concat([leak, b], axis=1)
            leaky_routing = tf.nn.softmax(leaky_logits, dim=1)        
            c = tf.split(leaky_routing, [1, output_capsule_num], axis=1)[1]
        else:
            c = softmax(b, 1)   
#        if i_activations is not None:
#            tf.transpose(tf.transpose(c, perm=[0,2,1]) * i_activations, perm=[0,2,1]) 
        outputs = squash_v1(K.batch_dot(c, u_hat_vecs, [2, 2]))
        if i < iterations - 1:
            b = b + K.batch_dot(outputs, u_hat_vecs, [2, 3])                                    
    poses = outputs 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations


In [ ]:
def vec_transformationByConv(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num):                            
    kernel = _get_weights_wrapper(
      name='weights', shape=[1, input_capsule_dim, output_capsule_dim*output_capsule_num], weights_decay_factor=0.0
    )
    tf.logging.info('poses: {}'.format(poses.get_shape()))   
    tf.logging.info('kernel: {}'.format(kernel.get_shape()))
    u_hat_vecs = keras.backend.conv1d(poses, kernel)
    u_hat_vecs = keras.backend.reshape(u_hat_vecs, (-1, input_capsule_num, output_capsule_num, output_capsule_dim))
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs


In [ ]:
def vec_transformationByMat(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num, shared=True):                        
    inputs_poses_shape = poses.get_shape().as_list()
    poses = poses[..., tf.newaxis, :]        
    poses = tf.tile(poses, [1, 1, output_capsule_num, 1])    
    if shared:
        kernel = _get_weights_wrapper(name='weights', shape=[1, 1, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], input_capsule_num, 1, 1, 1])
    else:
        kernel = _get_weights_wrapper(name='weights', shape=[1, input_capsule_num, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], 1, 1, 1, 1])
    tf.logging.info('poses: {}'.format(poses[...,tf.newaxis].get_shape()))   
    tf.logging.info('kernel: {}'.format(kernel.get_shape()))
    u_hat_vecs = tf.squeeze(tf.matmul(kernel, poses[...,tf.newaxis]),axis=-1)
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs

In [ ]:
def capsules_init(inputs, shape, strides, padding, pose_shape, add_bias, name):
    with tf.variable_scope(name):   
        poses = _conv2d_wrapper(
          inputs,
          shape=shape[0:-1] + [shape[-1] * pose_shape],
          strides=strides,
          padding=padding,
          add_bias=add_bias,
          activation_fn=None,
          name='pose_stacked'
        )        
        poses_shape = poses.get_shape().as_list()    
        poses = tf.reshape(poses, [-1, poses_shape[1], poses_shape[2], shape[-1], pose_shape])        
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, shape[-1]])    
        poses = squash_v1(poses, axis=-1)  
        activations = K.sqrt(K.sum(K.square(poses), axis=-1)) + beta_a        
        tf.logging.info("prim poses dimension:{}".format(poses.get_shape()))

    return poses, activations

In [ ]:
def capsule_fc_layer(nets, output_capsule_num, iterations, name):
    with tf.variable_scope(name):   
        poses, i_activations = nets
        input_pose_shape = poses.get_shape().as_list()

        u_hat_vecs = vec_transformationByConv(poses,input_pose_shape[-1], input_pose_shape[1],input_pose_shape[-1], output_capsule_num,)
        tf.logging.info('votes shape: {}'.format(u_hat_vecs.get_shape()))
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, output_capsule_num])
        poses, activations = routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations)
        
        tf.logging.info('capsule fc shape: {}'.format(poses.get_shape()))   
        
    return poses, activations

In [ ]:
def capsule_flatten(nets):
    poses, activations = nets
    input_pose_shape = poses.get_shape().as_list()
    
    poses = tf.reshape(poses, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3], input_pose_shape[-1]]) 
    activations = tf.reshape(activations, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3]])
    tf.logging.info("flatten poses dimension:{}".format(poses.get_shape()))
    tf.logging.info("flatten activations dimension:{}".format(activations.get_shape()))

    return poses, activations

In [ ]:
def capsule_conv_layer(nets, shape, strides, iterations, name):   
    with tf.variable_scope(name):              
        poses, i_activations = nets
        
        inputs_poses_shape = poses.get_shape().as_list()

        hk_offsets = [
          [(h_offset + k_offset) for k_offset in range(0, shape[0])] for h_offset in
          range(0, inputs_poses_shape[1] + 1 - shape[0], strides[1])
        ]
        wk_offsets = [
          [(w_offset + k_offset) for k_offset in range(0, shape[1])] for w_offset in
          range(0, inputs_poses_shape[2] + 1 - shape[1], strides[2])
        ]
    
        inputs_poses_patches = tf.transpose(
          tf.gather(
            tf.gather(
              poses, hk_offsets, axis=1, name='gather_poses_height_kernel'
            ), wk_offsets, axis=3, name='gather_poses_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5, 6], name='inputs_poses_patches'
        )
        tf.logging.info('i_poses_patches shape: {}'.format(inputs_poses_patches.get_shape()))
    
        inputs_poses_shape = inputs_poses_patches.get_shape().as_list()
        inputs_poses_patches = tf.reshape(inputs_poses_patches, [
                                -1, shape[0]*shape[1]*shape[2], inputs_poses_shape[-1]
                                ])

        i_activations_patches = tf.transpose(
          tf.gather(
            tf.gather(
              i_activations, hk_offsets, axis=1, name='gather_activations_height_kernel'
            ), wk_offsets, axis=3, name='gather_activations_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5], name='inputs_activations_patches'
        )
        tf.logging.info('i_activations_patches shape: {}'.format(i_activations_patches.get_shape()))
        i_activations_patches = tf.reshape(i_activations_patches, [
                                -1, shape[0]*shape[1]*shape[2]]
                                )
        u_hat_vecs = vec_transformationByConv(
                  inputs_poses_patches,
                  inputs_poses_shape[-1], shape[0]*shape[1]*shape[2],
                  inputs_poses_shape[-1], shape[3],
                  )
        tf.logging.info('capsule conv votes shape: {}'.format(u_hat_vecs.get_shape()))
    
        beta_a = _get_weights_wrapper(
                name='beta_a', shape=[1, shape[3]]
                )
        poses, activations = routing(u_hat_vecs, beta_a, iterations, shape[3], i_activations_patches)
        poses = tf.reshape(poses, [
                    inputs_poses_shape[0], inputs_poses_shape[1],
                    inputs_poses_shape[2], shape[3],
                    inputs_poses_shape[-1]]
                ) 
        activations = tf.reshape(activations, [
                    inputs_poses_shape[0],inputs_poses_shape[1],
                    inputs_poses_shape[2],shape[3]]
                ) 
        nets = poses, activations            
    tf.logging.info("capsule conv poses dimension:{}".format(poses.get_shape()))
    tf.logging.info("capsule conv activations dimension:{}".format(activations.get_shape()))
    return nets

In [ ]:
def capsule_model_A(X, num_classes):
    with tf.variable_scope('capsule_'+str(3)):   
        nets = _conv2d_wrapper(
                X, shape=[3, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
        tf.logging.info('output shape: {}'.format(nets.get_shape()))
        nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                             padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
        nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
        nets = capsule_flatten(nets)
        poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2') 
    return poses, activations

# **Load Data**

In [ ]:
import collections
import pickle
import gensim
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import re
import random
import sys
import os 
import pandas as pd
from sklearn.model_selection import train_test_split
from __future__ import print_function

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.layers import Dropout, Activation, Flatten,Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, Input, Dense, merge,Add
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.datasets import imdb
from keras import callbacks
from keras.utils import generic_utils
from keras.models import Model
from keras.optimizers import Adadelta
import time

In [ ]:
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

In [ ]:
DataTagged = pd.read_csv("/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/corpus/analyzed/lankadeepa_tagged_2.csv",";")
train_data, test_data = train_test_split(DataTagged, test_size=0.2, random_state=0)
comment_texts, comment_labels = text_preprocessing(train_data,test_data)

In [ ]:
DataTagged.head()

docid                                            comment     label
0  100204                               නියම සිංහල මහත්මයෙක්  POSITIVE
1  100204                                      අන්න මිනිස්සු  POSITIVE
2  100204  ගොවි මහත්තයාගේ ගෙවත්තේ ලබු වැලේ ගෙඩි 50ක් පමණ ...  POSITIVE
3  100204  වන සිවුපාවන් වැනි මිනිසුන් මද දෙවි දේවතාවුන් ව...  POSITIVE
4  100204                                        අයියෝ සල්ලි  NEGATIVE

In [ ]:
# prepare tokenizer
docs = comment_texts[0]+comment_texts[1]
# print(comment_texts[0])
# print(comment_texts[1])
# print(docs)
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
print(vocab_size)

18416


In [ ]:
print(t.word_index.items())

dict_items([('මේ', 1), ('අපේ', 2), ('නම්', 3), ('නැහැ', 4), ('වගේ', 5), ('ඒ', 6), ('ඔබ', 7), ('අපි', 8), ('ගැන', 9), ('එක', 10), ('කරන්න', 11), ('රටේ', 12), ('හරි', 13), ('තමයි', 14), ('මම', 15), ('අය', 16), ('කියලා', 17), ('කරන', 18), ('ඇති', 19), ('දැන්', 20), ('ඔය', 21), ('හොඳ', 22), ('ලංකාවේ', 23), ('ඉන්න', 24), ('මිනිස්සු', 25), ('රට', 26), ('නැති', 27), ('නිසා', 28), ('වෙන්න', 29), ('අනේ', 30), ('වැඩ', 31), ('වැඩක්', 32), ('ඔබට', 33), ('මේක', 34), ('වේවා', 35), ('දෙයක්', 36), ('තියෙන්නේ', 37), ('කියන්නේ', 38), ('ගන්න', 39), ('වලට', 40), ('මගේ', 41), ('එපා', 42), ('අද', 43), ('කරලා', 44), ('අපිට', 45), ('අයට', 46), ('මට', 47), ('රටට', 48), ('දේවල්', 49), ('කියල', 50), ('ලැබේවා', 51), ('සහ', 52), ('මේවා', 53), ('පුළුවන්', 54), ('වෙන', 55), ('වෙනවා', 56), ('කරනවා', 57), ('වෙලා', 58), ('හොද', 59), ('?', 60), ('සුබ', 61), ('යන්න', 62), ('හරිම', 63), ('ගොඩක්', 64), ('යන', 65), ('දෙන්න', 66), ('කළ', 67), ('නෑ', 68), ('කියන', 69), ('වල', 70), ('බව', 71), ('වඩා', 72), ('තව', 73), ('වෙන්නේ

In [ ]:
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[132, 104, 444, 18, 206, 1, 5, 4428, 11, 4429, 1532, 374, 76, 159, 32, 6, 9, 422, 16, 2087, 154, 9, 3204], [203, 25], [317, 179, 4430, 302, 7095, 180, 2511, 7096, 6, 4431, 681, 7097, 722, 47, 7098, 7099, 2512, 4432, 1533, 886, 771, 7100, 7101, 1336, 7102, 1337, 7103, 7104, 33, 149, 126], [7105, 267, 2513, 76, 7106, 209, 14, 74, 23, 64, 1338, 1339, 52, 2513, 772, 4433, 246, 56, 93, 2513, 602, 7107, 9, 375, 723, 568, 94], [7108, 2514, 376, 277, 23, 141, 3205, 7109, 17, 287, 7110, 832, 24, 107, 16, 18, 31, 569, 4434], [347, 4435, 59, 31], [1, 7111, 3, 23, 29, 145, 3206, 278, 773, 38, 7112, 1534, 2515, 7113, 318, 1535, 4436, 4], [4437, 682, 196, 52, 3207, 3208, 887, 3209, 1536, 3210, 3211, 3212, 887, 2088, 3213, 3214], [1770, 1771, 603, 66, 570], [4438, 4439, 3, 310, 4440, 5, 14, 4441], [2516, 3215, 62, 7114, 258, 84, 2089, 13, 7115], [1082, 7116, 2517, 333, 7117, 99, 36, 2518, 16, 173], [85, 16, 724, 4442, 7118, 2519, 7119, 279, 2520, 7120, 33, 2, 2090], [1340, 398, 4443, 23, 982, 7121, 

In [ ]:
print(docs[0])

['මිනිසුන්', 'තමන්ගේ', 'රැකියා', 'කරන', 'ගමන්', 'මේ', 'වගේ', 'කටයුතුත්', 'කරන්න', 'වෙලාව', 'හදාගෙන', 'කළේ', 'ලොකු', 'පින්', 'වැඩක්', 'ඒ', 'ගැන', 'නොදන්නා', 'අය', 'ලැබුණු', 'මුදල්', 'ගැන', 'අහනවා']


In [ ]:
print(encoded_docs[0][1])

104


In [ ]:
max_length = 155
padded_docs = pad_sequences(encoded_docs, maxlen=max_length)
print(padded_docs)

[[   0    0    0 ...  154    9 3204]
 [   0    0    0 ...    0  203   25]
 [   0    0    0 ...   33  149  126]
 ...
 [   0    0    0 ...  297 6381 3160]
 [   0    0    0 ...   51    7  242]
 [   0    0    0 ... 1131 6688   39]]


In [ ]:
keyed_fasttext_path = '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/word_embedding/fasttext/fasttext_keyed_vectors/keyed.kv'
word_vectors = KeyedVectors.load(keyed_fasttext_path, mmap='r')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print(len(word_vectors.vocab))
# for word in word_vectors.vocab.items():
#   print(word)


2137729


In [ ]:
embeddings_index = dict()
for word ,vocab_obj in word_vectors.vocab.items():
  embeddings_index[word]=word_vectors[word]

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
print(embedding_matrix.shape)

(18416, 300)


In [ ]:
embedding_matrix_path = '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/fasttetx_embedding_matrix'

In [ ]:
labels = comment_labels[0]+comment_labels[1]
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=0)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
maxlen = 155
max_features = vocab_size
embedding_dims = 300

# **Loss functions**

In [ ]:
def spread_loss(labels, activations, margin):
    activations_shape = activations.get_shape().as_list()
    mask_t = tf.equal(labels, 1)
    mask_i = tf.equal(labels, 0)    
    activations_t = tf.reshape(
      tf.boolean_mask(activations, mask_t), [activations_shape[0], 1]
    )    
    activations_i = tf.reshape(
      tf.boolean_mask(activations, mask_i), [activations_shape[0], activations_shape[1] - 1]
    )    
    gap_mit = tf.reduce_sum(tf.square(tf.nn.relu(margin - (activations_t - activations_i))))
    return gap_mit        

In [ ]:
def cross_entropy(y, preds):    
    y = tf.argmax(y, axis=1)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=preds, labels=y)                                               
    loss = tf.reduce_mean(loss) 
    return loss

In [ ]:
def margin_loss(y, preds):    
    y = tf.cast(y,tf.float32)
    loss = y * tf.square(tf.maximum(0., 0.9 - preds)) + \
        0.25 * (1.0 - y) * tf.square(tf.maximum(0., preds - 0.1))
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
#    loss = tf.reduce_mean(loss)    
    return loss

In [ ]:
def capsule_model_A(X, num_classes):
    with tf.variable_scope('capsule_'+str(3)):   
        nets = _conv2d_wrapper(
                X, shape=[3, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
        tf.logging.info('output shape: {}'.format(nets.get_shape()))
        nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                             padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
        nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
        nets = capsule_flatten(nets)
        poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2') 
    return poses, activations

In [ ]:
def capsule_model_B(X, num_classes):
    poses_list = []
    for _, ngram in enumerate([3,4,5]):
        with tf.variable_scope('capsule_'+str(ngram)): 
            nets = _conv2d_wrapper(
                X, shape=[ngram, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
            tf.logging.info('output shape: {}'.format(nets.get_shape()))
            nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                                 padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
            nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
            nets = capsule_flatten(nets)
            poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2')
            poses_list.append(poses)
    
    poses = tf.reduce_mean(tf.convert_to_tensor(poses_list), axis=0) 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations

# **Training process**

In [ ]:
class Args:
  embedding_type = "static"
  dataset = "reuters_multilabel_dataset"
  loss_type = "margin_loss"
  model_type = "capsule-A"
  has_test = 1
  has_dev = 1
  num_epochs = 100
  batch_size = 25
  use_orphan = True
  use_leaky = False
  learning_rate = 0.001
  margin = 0.2
  num_classes = 2
  vocab_size = 18416
  vec_size = 300
  max_sent = 155


In [ ]:
args = Args()

In [ ]:
len(X_train[0])

155

In [ ]:

X = tf.placeholder(tf.int32, [args.batch_size, args.max_sent], name="input_x")
y = tf.placeholder(tf.int64, [args.batch_size, args.num_classes], name="input_y")
is_training = tf.placeholder_with_default(False, shape=())    
learning_rate = tf.placeholder(dtype='float32') 
margin = tf.placeholder(shape=(),dtype='float32') 

l2_loss = tf.constant(0.0)

In [ ]:
w2v = np.array(embedding_matrix,dtype=np.float32)

In [ ]:
W1 = tf.Variable(w2v, trainable = False)
X_embedding = tf.nn.embedding_lookup(W1, X)
X_embedding = X_embedding[...,tf.newaxis] 

In [ ]:
tf.logging.info("input dimension:{}".format(X_embedding.get_shape()))

INFO:tensorflow:input dimension:(25, 155, 300, 1)


In [ ]:
X_embedding.shape

TensorShape([Dimension(25), Dimension(155), Dimension(300), Dimension(1)])

In [ ]:
# poses, activations = capsule_model_A(X_embedding, args.num_classes)   
poses, activations = capsule_model_B(X_embedding, args.num_classes)   

INFO:tensorflow:output shape: (25, 77, 1, 32)
INFO:tensorflow:prim poses dimension:(25, 77, 1, 16, 16)
INFO:tensorflow:i_poses_patches shape: (25, 75, 1, 3, 1, 16, 16)
INFO:tensorflow:i_activations_patches shape: (25, 75, 1, 3, 1, 16)
INFO:tensorflow:poses: (1875, 48, 16)
INFO:tensorflow:kernel: (1, 16, 256)
INFO:tensorflow:capsule conv votes shape: (1875, 16, 48, 16)
INFO:tensorflow:capsule conv poses dimension:(25, 75, 1, 16, 16)
INFO:tensorflow:capsule conv activations dimension:(25, 75, 1, 16)
INFO:tensorflow:flatten poses dimension:(25, 1200, 16)
INFO:tensorflow:flatten activations dimension:(25, 1200)
INFO:tensorflow:poses: (25, 1200, 16)
INFO:tensorflow:kernel: (1, 16, 32)
INFO:tensorflow:votes shape: (25, 2, 1200, 16)
INFO:tensorflow:capsule fc shape: (25, 2, 16)
INFO:tensorflow:output shape: (25, 76, 1, 32)
INFO:tensorflow:prim poses dimension:(25, 76, 1, 16, 16)
INFO:tensorflow:i_poses_patches shape: (25, 74, 1, 3, 1, 16, 16)
INFO:tensorflow:i_activations_patches shape: (25, 

In [ ]:
print("well done Piyumal")

well done Piyumal


In [ ]:
loss = margin_loss(y, activations)

In [ ]:
y_pred = tf.argmax(activations, axis=1, name="y_proba")    
correct = tf.equal(tf.argmax(y, axis=1), y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)   
training_op = optimizer.minimize(loss, name="training_op")
gradients, variables = zip(*optimizer.compute_gradients(loss))

In [ ]:
with tf.device('/cpu:0'):
    global_step = tf.train.get_or_create_global_step()

In [ ]:
args.max_sent = 200
threshold = 0.5

In [ ]:
grad_check = [tf.check_numerics(g, message='Gradient NaN Found!')
              for g in gradients if g is not None] + [tf.check_numerics(loss, message='Loss NaN Found')]
with tf.control_dependencies(grad_check):
    training_op = optimizer.apply_gradients(zip(gradients, variables), global_step=global_step)      

In [ ]:
sess = tf.InteractiveSession()
from keras import utils

In [ ]:
class BatchGenerator(object):
    """Generate and hold batches."""
    def __init__(self, dataset,label, batch_size,input_size, is_shuffle=True):
      self._dataset = dataset
      self._label = label
      self._batch_size = batch_size    
      self._cursor = 0      
      self._input_size = input_size      
      
      if is_shuffle:
          index = np.arange(len(self._dataset))
          np.random.shuffle(index)
          self._dataset = np.array(self._dataset)[index]
          self._label = np.array(self._label)[index]
      else:
          self._dataset = np.array(self._dataset)
          self._label = np.array(self._label)
    def next(self):
      if self._cursor + self._batch_size > len(self._dataset):
          self._cursor = 0
      """Generate a single batch from the current cursor position in the data."""      
      batch_x = self._dataset[self._cursor : self._cursor + self._batch_size,:]
      batch_y = self._label[self._cursor : self._cursor + self._batch_size]
      self._cursor += self._batch_size
      return batch_x, batch_y

In [ ]:
n_iterations_per_epoch = len(X_train) // args.batch_size
n_iterations_test = len(X_test) // args.batch_size
n_iterations_dev = len(X_test) // args.batch_size    

mr_train = BatchGenerator(X_train,y_train, args.batch_size, 0)    
mr_dev = BatchGenerator(X_test,y_test, args.batch_size, 0)
mr_test = BatchGenerator(X_test,X_test, args.batch_size, 0, is_shuffle=False)

best_model = None
best_epoch = 0
best_acc_val = 0.

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)     

lr = args.learning_rate
m = args.margin

In [ ]:
for epoch in range(args.num_epochs):
    for iteration in range(1, n_iterations_per_epoch + 1):                
        X_batch, y_batch = mr_train.next()     
        y_batch = utils.to_categorical(y_batch, args.num_classes)        
        _, loss_train, probs, capsule_pose = sess.run(
            [training_op, loss, activations, poses],
            feed_dict={X: X_batch[:,:args.max_sent],
                       y: y_batch,
                       is_training: True,
                       learning_rate:lr,
                       margin:m})        
        print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                  iteration, n_iterations_per_epoch,
                  iteration * 100 / n_iterations_per_epoch,
                  loss_train),
              end="")                        
    loss_vals, acc_vals = [], []
    for iteration in range(1, n_iterations_dev + 1):
        X_batch, y_batch = mr_dev.next()            
        y_batch = utils.to_categorical(y_batch, args.num_classes)
        loss_val, acc_val = sess.run(
                [loss, accuracy],
                feed_dict={X: X_batch[:,:args.max_sent],
                           y: y_batch,
                           is_training: False,
                           margin:m})
        loss_vals.append(loss_val)
        acc_vals.append(acc_val)
    loss_val, acc_val = np.mean(loss_vals), np.mean(acc_vals)    
    print("\rEpoch: {}  Val accuracy: {:.1f}%  Loss: {:.4f}".format(
        epoch + 1, acc_val * 100, loss_val))
               
    # preds_list, y_list = [], []
    # for iteration in range(1, n_iterations_test + 1):
    #     X_batch, y_batch = mr_test.next()             
    #     probs = sess.run([activations],
    #             feed_dict={X:X_batch[:,:args.max_sent],
    #                        is_training: False})
    #     preds_list = preds_list + probs[0].tolist()
    #     y_list = y_list + y_batch.tolist()
        
    # y_list = np.array(y_list)
    # preds_probs = np.array(preds_list)                
    # preds_probs[np.where( preds_probs >= threshold )] = 1.0
    # preds_probs[np.where( preds_probs < threshold )] = 0.0 
    # print(y_list)
    # print(preds_probs)
    
    # #[precision, recall, F1, support] = precision_recall_fscore_support(y_list, preds_probs, average='samples')
    # acc = accuracy_score(y_list, preds_probs)

    # print ('\rER: %.3f' % acc, 'Precision: %.3f' % precision, 'Recall: %.3f' % recall, 'F1: %.3f' % F1)  
    # # if args.model_type == 'CNN' or args.model_type == 'KIMCNN':
    # #     lr = max(1e-6, lr * 0.8)
    if args.loss_type == 'margin_loss':    
        m = min(0.9, m + 0.1)

Epoch: 1  Val accuracy: 83.5%  Loss: 0.0735
Epoch: 2  Val accuracy: 82.7%  Loss: 0.0725
Epoch: 3  Val accuracy: 84.3%  Loss: 0.0718
Epoch: 4  Val accuracy: 83.1%  Loss: 0.0769
Epoch: 5  Val accuracy: 83.6%  Loss: 0.0751
Epoch: 6  Val accuracy: 83.8%  Loss: 0.0749
Epoch: 7  Val accuracy: 83.5%  Loss: 0.0829
Epoch: 8  Val accuracy: 82.9%  Loss: 0.0950
Epoch: 9  Val accuracy: 83.0%  Loss: 0.0855
Epoch: 10  Val accuracy: 83.1%  Loss: 0.0833
Epoch: 11  Val accuracy: 84.0%  Loss: 0.0783
Iteration: 83/160 (51.9%)  Loss: 0.02596

KeyboardInterrupt: ignored

In [ ]:
y_list

In [ ]:
best_acc_val